In [1]:
import pandas as pd
import numpy as np 
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect
from sqlalchemy import MetaData
import sqlite3
import datetime

In [2]:
engine = create_engine('sqlite:///northwind.db') #Creacion de la conexion de la base de datos

In [3]:
frameAlbums = pd.read_sql_query('''SELECT  * from Employees''', con=engine.connect())
frameAlbums

,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Ms.,1948-12-08 00:00:00,1992-05-01 00:00:00,507 - 20th Ave. E.\r\nApt. 2A,Seattle,WA,98122,USA,(206) 555-9857,5467,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Education includes a BA in psychology from Col...,2.0,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1952-02-19 00:00:00,1992-08-14 00:00:00,908 W. Capital Way,Tacoma,WA,98401,USA,(206) 555-9482,3457,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Andrew received his BTS commercial in 1974 and...,NaN,http://accweb/emmployees/fuller.bmp
2,3,Leverling,Janet,Sales Representative,Ms.,1963-08-30 00:00:00,1992-04-01 00:00:00,722 Moss Bay Blvd.,Kirkland,WA,98033,USA,(206) 555-3412,3355,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Janet has a BS degree in chemistry from Boston...,2.0,http://accweb/emmployees/leverling.bmp
3,4,Peacock,Margaret,Sales Representative,Mrs.,1937-09-19 00:00:00,1993-05-03 00:00:00,4110 Old Redmond Rd.,Redmond,WA,98052,USA,(206) 555-8122,5176,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Margaret holds a BA in English literature from...,2.0,http://accweb/emmployees/peacock.bmp
4,5,Buchanan,Steven,Sales Manager,Mr.,1955-03-04 00:00:00,1993-10-17 00:00:00,14 Garrett Hill,London,None,SW1 8JR,UK,(71) 555-4848,3453,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Steven Buchanan graduated from St. Andrews Uni...,2.0,http://accweb/emmployees/buchanan.bmp
5,6,Suyama,Michael,Sales Representative,Mr.,1963-07-02 00:00:00,1993-10-17 00:00:00,Coventry House\r\nMiner Rd.,London,None,EC2 7JR,UK,(71) 555-7773,428,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Michael is a graduate of Sussex University (MA...,5.0,http://accweb/emmployees/davolio.bmp
6,7,King,Robert,Sales Representative,Mr.,1960-05-29 00:00:00,1994-01-02 00:00:00,Edgeham Hollow\r\nWinchester Way,London,None,RG1 9SP,UK,(71) 555-5598,465,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Robert King served in the Peace Corps and trav...,5.0,http://accweb/emmployees/davolio.bmp
7,8,Callahan,Laura,Inside Sales Coordinator,Ms.,1958-01-09 00:00:00,1994-03-05 00:00:00,4726 - 11th Ave. N.E.,Seattle,WA,98105,USA,(206) 555-1189,2344,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Laura received a BA in psychology from the Uni...,2.0,http://accweb/emmployees/davolio.bmp
8,9,Dodsworth,Anne,Sales Representative,Ms.,1966-01-27 00:00:00,1994-11-15 00:00:00,7 Houndstooth Rd.,London,None,WG2 7LT,UK,(71) 555-4444,452,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Anne has a BA degree in English from St. Lawre...,5.0,http://accweb/emmployees/davolio.bmp


In [4]:
metadata = MetaData()
metadata.create_all(engine)
inspector = inspect(engine)
inspector.get_columns('Customers')

[{'name': 'CustomerID',
  'type': CHAR(length=5),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'CompanyName',
  'type': NVARCHAR(length=40),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'ContactName',
  'type': NVARCHAR(length=30),
  'nullable': True,
  'default': 'NULL',
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'ContactTitle',
  'type': NVARCHAR(length=30),
  'nullable': True,
  'default': 'NULL',
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Address',
  'type': NVARCHAR(length=60),
  'nullable': True,
  'default': 'NULL',
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'City',
  'type': NVARCHAR(length=15),
  'nullable': True,
  'default': 'NULL',
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Region',
  'type': NVARCHAR(length=15),
  'nullable': True,
  'default': 'NULL',
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name'

In [5]:
df_customer = pd.read_sql_query(
"""
SELECT 
    CompanyName,  
    ContactName,  
    ContactTitle, 
    Address,      
    City,         
    Region,     
    ifnull(PostalCode ,'No Asigando')as PostalCode,
    replace(replace(Country,"UK","United Kingdom"),"USA","The United States of America") as Country, 
    Phone,
    Fax            
FROM Customers;
"""
,con=engine.connect())
df_customer.head(8)


,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,United Kingdom,(171) 555-7788,(171) 555-6750
4,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
5,Blauer See Delikatessen,Hanna Moos,Sales Representative,Forsterstr. 57,Mannheim,None,68306,Germany,0621-08460,0621-08924
6,Blondesddsl père et fils,Frédérique Citeaux,Marketing Manager,"24, place Kléber",Strasbourg,None,67000,France,88.60.15.31,88.60.15.32
7,Bólido Comidas preparadas,Martín Sommer,Owner,"C/ Araquil, 67",Madrid,None,28023,Spain,(91) 555 22 82,(91) 555 91 99


In [6]:
df_employe = pd.read_sql_query(
"""
SELECT 
    LastName,
    FirstName,       
    Title,         
    TitleOfCourtesy, 
    BirthDate  ,    
    HireDate ,      
    Address,        
    City  ,         
    Region,        
    PostalCode,      
    replace(replace(Country,"UK","United Kingdom"),"USA","The United States of America") as Country,        
    HomePhone,      
    Extension,       
    ifnull(Photo ,'No Asigando')as Photo,             
    Notes,          
    ifnull(ReportsTo ,'No Asigando')as ReportsTo,          
    PhotoPath             
FROM Employees;
"""
,con=engine.connect())
df_employe.head(4)

,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,Davolio,Nancy,Sales Representative,Ms.,1948-12-08 00:00:00,1992-05-01 00:00:00,507 - 20th Ave. E.\r\nApt. 2A,Seattle,WA,98122,The United States of America,(206) 555-9857,5467,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Education includes a BA in psychology from Col...,2,http://accweb/emmployees/davolio.bmp
1,Fuller,Andrew,"Vice President, Sales",Dr.,1952-02-19 00:00:00,1992-08-14 00:00:00,908 W. Capital Way,Tacoma,WA,98401,The United States of America,(206) 555-9482,3457,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Andrew received his BTS commercial in 1974 and...,No Asigando,http://accweb/emmployees/fuller.bmp
2,Leverling,Janet,Sales Representative,Ms.,1963-08-30 00:00:00,1992-04-01 00:00:00,722 Moss Bay Blvd.,Kirkland,WA,98033,The United States of America,(206) 555-3412,3355,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Janet has a BS degree in chemistry from Boston...,2,http://accweb/emmployees/leverling.bmp
3,Peacock,Margaret,Sales Representative,Mrs.,1937-09-19 00:00:00,1993-05-03 00:00:00,4110 Old Redmond Rd.,Redmond,WA,98052,The United States of America,(206) 555-8122,5176,b'\x15\x1c/\x00\x02\x00\x00\x00\r\x00\x0e\x00\...,Margaret holds a BA in English literature from...,2,http://accweb/emmployees/peacock.bmp


In [7]:
df_location = pd.read_sql_query(
"""
SELECT  
    ShipName,
    ShipAddress,   
    ShipCity  ,    
    ShipRegion ,    
    ifnull(ShipPostalCode ,'No Asigando')as ShipPostalCode,     
    replace(replace(ShipCountry,"UK","United Kingdom"),"USA","The United States of America") as ShipCountry              
                   
FROM Orders;
"""
,con=engine.connect())
df_location.head(4)

,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France


In [8]:
df_produc = pd.read_sql_query(
"""
SELECT 
   
    ProductName,
    SupplierId,
    CategoryId,                              
    QuantityPerUnit, 
    UnitPrice,      
    UnitsInStock,    
    UnitsOnOrder,   
    ReorderLevel,   
    Discontinued           
FROM Products;
"""
,con=engine.connect())
df_produc.head(4)

,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,0
1,Chang,1,1,24 - 12 oz bottles,19.0,17,40,25,0
2,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0,13,70,25,0
3,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.0,53,0,0,0


In [9]:
df_shipper = pd.read_sql_query(
"""
SELECT ShipperID, 
    CompanyName,
    Phone           
FROM Shippers;
"""
,con=engine.connect())
df_shipper.head(4)

,ShipperID,CompanyName,Phone
0,1,Speedy Express,(503) 555-9831
1,2,United Package,(503) 555-3199
2,3,Federal Shipping,(503) 555-9931


In [10]:
df_supplier = pd.read_sql_query(
"""
SELECT 
                
    CompanyName ,
    ContactName  ,
    ContactTitle ,
    Address      ,
    City        ,
    Region      ,
    PostalCode  ,
    replace(replace(Country,"UK","United Kingdom"),"USA","The United States of America") as Country ,           
    ifnull(Fax ,'No Asigando')as Fax,
    ifnull(HomePage ,'No Asigando')as HomePage,         
    Phone    
    FROM Suppliers;
"""
,con=engine.connect())
df_supplier.head(4)

,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Fax,HomePage,Phone
0,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,United Kingdom,No Asigando,No Asigando,(171) 555-2222
1,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,The United States of America,No Asigando,#CAJUN.HTM#,(100) 555-4822
2,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,The United States of America,(313) 555-3349,No Asigando,(313) 555-5735
3,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,None,100,Japan,No Asigando,No Asigando,(03) 3555-5011


In [11]:
fecha_inicio='2009-01-01'
fecha_fin='2030-12-31'
dataFrameDate = pd.DataFrame({'fecha': pd.date_range(fecha_inicio, fecha_fin)})
dataFrameDate['año'] = dataFrameDate.fecha.dt.year
dataFrameDate['mes'] = dataFrameDate.fecha.dt.month
dataFrameDate['dia'] = dataFrameDate.fecha.dt.day
dataFrameDate['nombre_dia'] = dataFrameDate.fecha.dt.day_name()
dataFrameDate['dia_semana'] = dataFrameDate.fecha.dt.dayofweek
dataFrameDate['semana'] = dataFrameDate.fecha.dt.weekofyear
dataFrameDate['trimestre'] = dataFrameDate.fecha.dt.quarter
dataFrameDate = dataFrameDate[['fecha', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']] 
dataFrameDate

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


,fecha,año,mes,dia,nombre_dia,dia_semana,semana,trimestre
0,2009-01-01,2009,1,1,Thursday,3,1,1
1,2009-01-02,2009,1,2,Friday,4,1,1
2,2009-01-03,2009,1,3,Saturday,5,1,1
3,2009-01-04,2009,1,4,Sunday,6,1,1
4,2009-01-05,2009,1,5,Monday,0,2,1
...,...,...,...,...,...,...,...,...
8030,2030-12-27,2030,12,27,Friday,4,52,4
8031,2030-12-28,2030,12,28,Saturday,5,52,4
8032,2030-12-29,2030,12,29,Sunday,6,52,4
8033,2030-12-30,2030,12,30,Monday,0,1,4


In [12]:
TablaHechos = pd.read_sql_query(
"""   
SELECT OrderDetail.OrdersDetailID AS OrderID,Customers.CustomerID AS CustomerID, 
Employees.EmployeeID AS EmployeeId, Orders.OrderID AS LocationID, Orders.OrderDate,
Products.ProductID AS ProductID, Shippers.ShipperID AS ShipperID, Suppliers.SupplierID AS SupplierID, 
OrderDetail.UnitPrice
FROM OrderDetail
 INNER JOIN Orders  ON Orders.OrdersID = OrderDetails.OrderID
INNER JOIN  Customers ON Customers.CustomerID = Orders.CustomerID 
INNER JOIN Employees  ON Employees.EmployeeID = Orders.EmployeeID
INNER JOIN Products  ON Products.ProductID = OrderDetail.ProductID
 INNER JOIN Shippers  ON Shippers.ShipperID = Orders.ShipVia 
 INNER JOIN Suppliers ON Suppliers.SupplierID = Products.SupplierID ;
""" ,con=engine.connect())
TablaHechos.head(4)

OperationalError: (sqlite3.OperationalError) no such table: OrderDetail
[SQL:    
SELECT OrderDetail.OrdersDetailID AS OrderID,Customers.CustomerID AS CustomerID, 
Employees.EmployeeID AS EmployeeId, Orders.OrderID AS LocationID, Orders.OrderDate,
Products.ProductID AS ProductID, Shippers.ShipperID AS ShipperID, Suppliers.SupplierID AS SupplierID, 
OrderDetail.UnitPrice
FROM OrderDetail
 INNER JOIN Orders  ON Orders.OrdersID = OrderDetails.OrderID
INNER JOIN  Customers ON Customers.CustomerID = Orders.CustomerID 
INNER JOIN Employees  ON Employees.EmployeeID = Orders.EmployeeID
INNER JOIN Products  ON Products.ProductID = OrderDetail.ProductID
 INNER JOIN Shippers  ON Shippers.ShipperID = Orders.ShipVia 
 INNER JOIN Suppliers ON Suppliers.SupplierID = Products.SupplierID ;
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [13]:
con2 = create_engine('sqlite:///DW_Northwind_Estrella.sqlite')
con2

Engine(sqlite:///DW_Northwind_Estrella.sqlite)

In [ ]:
df_customer.to_sql("dim_customers",con2, if_exists='append',index = False, )

In [ ]:
def updateData(name_table,data_db,name_columns):
	print(name_columns)
	aux_values='?'
	aux_data=[]
	aux_columns=name_columns[0]
	aux_data.append(list(data_db[name_columns[0]]))
	entities=[]
	aux_entities=[]
	for i in range(len(name_columns)-1):
		aux_values+=',?'
		aux_columns+=','+name_columns[i+1]
		aux_data.append(list(data_db[name_columns[i+1]]))
	for i in range(len(aux_data[0])):
		for j in range(len(aux_data)):
			aux_entities.append(aux_data[j][i])
		entities.append(aux_entities)
		aux_entities=[]
	with con2.connect() as con:
		for i in entities:
			con.execute('INSERT INTO '+name_table+'('+aux_columns+') VALUES('+aux_values+')', i)
	con.close()

In [ ]:
updateData('dim_customer',df_customer,list(df_customer.columns.values))
updateData('dim_location',df_location,list(df_location.columns.values))
updateData('dim_employee',df_employe ,list(df_employe.columns.values))
updateData('dim_product',df_produc,list(df_produc.columns.values))
updateData('dim_shipper',df_shipper,list(df_shipper.columns.values))
updateData('dim_supplier',df_supplier,list(df_supplier.columns.values))
dataFrameDate.to_sql(name='dim_time', con=engine.connect(), if_exists='append',index=False)
print()
updateData('factsales_order',TablaHechos,list(TablaHechos.columns.values))